In [1]:
using Distributions, CSV, DataFrames, ProgressMeter
include("../../utils_sg.jl")

### Parameters

In [2]:
S0 = 100.0; v0 = 0.4; ρ = 0.8; T = 1.0; b = 0.04; β = 0.25; σ₀ = 0.25; η = 0.5; α = 0.3;
R = 0.07; r = 0.006; K1 = 95.0; K2 = 105.0;
slvm = HypHypSLV(S0::Float64, v0::Float64, ρ::Float64, b::Float64, β::Float64, σ₀::Float64, η::Float64, α::Float64);
Π = [1.0 ρ; ρ 1.0]; cholΠ=cholesky(Π).L

drift(x) = [slvm.ω(abs(x[1]),abs(x[2])), slvm.μ(abs(x[2]))]
diffusion(x) = Diagonal(
        [slvm.m(abs(x[2]))*slvm.Γ(abs(x[1])), slvm.σ(abs(x[2]))]
        )*cholΠ
driver(t, x, y, z) = (
    -r*max(y-z'*inv(diffusion(x))*x, 0.0)
    -R*min(y-z'*inv(diffusion(x))*x, 0.0)
    -z'*inv(diffusion(x))*drift(x)
)
terminal(x) = max(x[1]-K1,0.0)-2.0*max(x[1]-K2,0.0)


bsde = BSDE(T, [S0, v0], drift, diffusion, driver, terminal);

## Experiment

In [3]:
Nₜs = [10, 20, 50, 100, 200, 500]
header = vcat(["scheme", "measurement_type"], string.(Nₜs))
schemes = [
    [HochOst4(krylov=true, m=100), true],
    [LawsonEuler(krylov=true, m=100), true],
    [NorsettEuler(krylov=true, m=100), true],
    [ETDRK2(krylov=true, m=100), true],
    [ETDRK3(krylov=true, m=100), true],
    [ETDRK4(krylov=true, m=100), true],
    [HochOst4(krylov=true, m=100), true]
    # [DP5(), false],
    # [RadauIIA5(), false]
]

7-element Vector{Vector{Any}}:
 [HochOst4{0, true, Val{:forward}, true, nothing}(true, 100, 0), true]
 [LawsonEuler{0, true, Val{:forward}, true, nothing}(true, 100, 0), true]
 [NorsettEuler{0, true, Val{:forward}, true, nothing}(true, 100, 0), true]
 [ETDRK2{0, true, Val{:forward}, true, nothing}(true, 100, 0), true]
 [ETDRK3{0, true, Val{:forward}, true, nothing}(true, 100, 0), true]
 [ETDRK4{0, true, Val{:forward}, true, nothing}(true, 100, 0), true]
 [HochOst4{0, true, Val{:forward}, true, nothing}(true, 100, 0), true]

In [4]:
### designing grids
d = 2
### Parameters for Tavella-Randall grids
g = 5.0
g₁=g
g₂=g

lefts = [bsde.X0[1]/100, bsde.X0[2]/15];
rights = [2*bsde.X0[1], 2*bsde.X0[2]].-lefts;
centers = [100.0, 0.4];

In [5]:
function solveProb(mind, bsde, Nₜ, scheme)
    d = length(mind)
    scaled_centers=(centers.-(rights.+lefts)./2.0)./((rights.-lefts)./2.0);
    _p = [rescale(eq_nodes(mind[dim]), scaled_centers[dim], lefts[dim], rights[dim]) for dim in 1:d]
    Δₗ = [pp[2]-pp[1] for pp in _p]
    Δᵣ = [pp[end]-pp[end-1] for pp in _p]
    Nₗ = [length(pp)÷2 for pp in _p]
    Nᵣ = [length(pp)÷2 for pp in _p]

    ratioₗ = Δₗ.*Nₗ./(Δₗ[1]*Nₗ[1])
    ratioᵣ = Δᵣ.*Nᵣ./(Δᵣ[1]*Nᵣ[1])
    grids = Array{AbstractGrid,1}(undef, d)
    grids[1] = TavellaRandallGrid(g₁, g₂, _p[1][1], _p[1][length(_p[1])÷2+1], _p[1][end], length(_p[1])÷2, length(_p[1])÷2)
    grids[2] = Grid1D(_p[2], length(_p[2]), length(_p[2])÷2, length(_p[2])÷2)
    res = MethodOfLines(bsde, grids, Nₜ, scheme, true)
    return [res[1], grids]
end

solveProb (generic function with 1 method)

In [6]:
dfs = Array{DataFrame, 1}(undef, length(schemes))
@showprogress for (snum, attr) in enumerate(schemes)
    scheme = attr[1]
    EXPINT = attr[2]
    schemename = split(split(string(scheme), '{')[1], '(')[1]
    dfs[snum] = DataFrame([[],[],[],[],[],[],[],[]], header)
    @showprogress for q in [7,8,9,10]
        print(string("q=", q,":\n")); flush(stdout)
        sol95 = zeros(Float64, length(Nₜs));     
        sol100 = zeros(Float64, length(Nₜs)); 
        sol105 = zeros(Float64, length(Nₜs)); 
        runtimes = zeros(Float64, length(Nₜs))
        spind = sparse_indices(q, d)
        @showprogress for (ind, Nₜ) in enumerate(Nₜs)
            print(string("Nₜ=", Nₜ,":\n")); flush(stdout)
            timestep = max(Nₜ÷100,1)
            exc_start = time()
            results = [solveProb(mind, bsde, Nₜ, scheme) for mind in spind]
            data = [dat[1] for dat in results]
            grids = [dat[2] for dat in results]
            if all([size(dat)[2] == Nₜ+1 for dat in data])
                sg_interp = SparseGridInterpolation(data, grids, spind, q, d, Nₜ, lefts, rights, centers)
                exc_stop = time()
                sol95[ind] = sg_interp.Interpolant(Nₜ+1, [95.0,0.4])
                sol100[ind] = sg_interp.Interpolant(Nₜ+1, [100.0,0.4])
                sol105[ind] = sg_interp.Interpolant(Nₜ+1, [105.0,0.4])
            else
                exc_stop = time()
                sol95[ind] = Inf
                sol100[ind] = Inf
                sol105[ind] = Inf
            end
            runtimes[ind] = exc_stop - exc_start
            print(string("Y0(95,0.4)=", sol95[ind],":\n")); flush(stdout)
            print(string("Y0(100,0.4)=", sol100[ind],":\n")); flush(stdout)
            print(string("Y0(105,0.4)=", sol105[ind],":\n")); flush(stdout)
            print(string("Runtime[s]=", runtimes[ind],":\n")); flush(stdout)
        end
        row_95 = vcat([string(q), "Y0(95,0.4)"], string.(sol95)); push!(dfs[snum],row_95);
        row_100 = vcat([string(q), "Y0(100,0.4)"], string.(sol100)); push!(dfs[snum],row_100);
        row_105 = vcat([string(q), "Y0(105,0.4)"], string.(sol105)); push!(dfs[snum],row_105);
        row_run = vcat([string(q), "Runtime[s]"], string.(runtimes)); push!(dfs[snum],row_run);
    end
    dfs[snum] |> CSV.write(string(schemename, ".csv"))
end

q=7:
Nₜ=10:
Y0(95,0.4)=4.408377665236383:
Y0(100,0.4)=5.813014880386875:
Y0(105,0.4)=5.622148911457231:
Runtime[s]=10.617328882217407:
Nₜ=20:
Y0(95,0.4)=4.408076159362286:
Y0(100,0.4)=5.812748504212447:
Y0(105,0.4)=5.621883270362696:
Runtime[s]=2.588628053665161:


Progress:  33%|█████████████▋                           |  ETA: 0:00:27

Nₜ=50:
Y0(95,0.4)=4.408004921772568:
Y0(100,0.4)=5.812673572330828:
Y0(105,0.4)=5.6218107501694305:
Runtime[s]=5.963331937789917:


Progress:  50%|████████████████████▌                    |  ETA: 0:00:20

Nₜ=100:
Y0(95,0.4)=4.408004272222177:
Y0(100,0.4)=5.812682570668816:
Y0(105,0.4)=5.621816640167103:
Runtime[s]=11.494948148727417:


...

Progress:  67%|███████████████████████████▍             |  ETA: 0:01:31

Nₜ=200:
Y0(95,0.4)=4.4046769462916755:
Y0(100,0.4)=5.819844824797343:
Y0(105,0.4)=5.637129185620383:
Runtime[s]=195.72251915931702:


Progress:  83%|██████████████████████████████████▏      |  ETA: 0:01:16

Nₜ=500:
Y0(95,0.4)=4.404667116950568:
Y0(100,0.4)=5.819832078755204:
Y0(105,0.4)=5.637117699849672:
Runtime[s]=482.85816192626953:


Progress: 100%|█████████████████████████████████████████| Time: 0:14:21
Progress: 100%|█████████████████████████████████████████| Time: 0:25:40
Progress: 100%|█████████████████████████████████████████| Time: 2:01:09
